# typing, attr.s, dataclasses - k čemu je to dobré

**Abstrakt**

Python 3.6 přinesl podporu 'type hints/annotations' zavádějící některé vlastnosti staticky typovaných jazyků. Ukážeme si proč je to dobré a jak zůstává zachována kompatibilita s duck-typingem. Dále krátce představím knihovnu 'attr.s' a porovnám jí s možnostmi 'dataclasses' zavedenými v Python 3.7. Přehledová přednáška vychází z aplikace uvedených technologií při vývoji workflow systému pro high performance computing.

**Odvozeno od:**

[Carl Meyer (Instagram): Type-checked Python in the real world, PyCon 2018](https://www.youtube.com/watch?v=pMgmKJyWKn8)



## Zdrojový kód = zápis programu **čitelný lidmi**




## duck typing / dynamic typing

- kompaktní kód (no boilerplate)
- redukce informačního šumu
- rychlejší vývoj (unit tests, refacotring):
    - žádná dlouhá kompilace 
    - méně času stráveného laděním formálních chyb

## static typing

- lepší informace o předávaných datech
- lepší definice API, důležité pro velké projekty
- méně chyb, dřívější odhalení  
- rychlejší běh 

 

In [ ]:
def process(self, items):
    for item in items:
        self.append(item.value.id)

- duck-typing = typ 'items' dán implicitně kódem
- pro složitější funkce časově náročné
- neodhalené chyby při použití funkce
- problematický refactoring (ve velkém projektu)

In [ ]:
def process(self, items):
    for item in items:
        self.append(item.value.id)

`items` je:
- sekvence objektů:
- majících atribut *value*, jehož typ 
- má atribut *id*

In [ ]:
from typing import Sequence
from .model import Item

def process(self, items : Sequence[Item]) -> None:
    for item in items:
        self.append(item.value.id)

`items` je:
- sekvence objektů:
- majících atribut *value*, jehož typ 
- má atribut *id*

## Jak používat 'type annotations'?
`typecheck` = IPython magie, která spustí 'mypy' na kód v buňce

In [12]:
%%typecheck

from typing import *

def square( x: float) -> float:
    return x**2

def main() -> None:
    square(1)
    square(1.0)
    square("one")
    square(1) + "one"

def no_type_check():
    square("one")

<string>:11: error: Argument 1 to "square" has incompatible type "str"; expected "float"
<string>:12: error: Unsupported operand types for + ("float" and "str")



## Odvozování typů

In [14]:
%%typecheck
from typing import *

class Photo:
    def __init__(self, width:int, height: int):
        self.width, self.height = width, height
    
    def dimensions(self) -> Tuple[str, str]:
        return (self.width, self.height)

<string>:9: error: Incompatible return value type (got "Tuple[int, int]", expected "Tuple[str, str]")



In [ ]:
## Homogenní listy

In [17]:
%%typecheck
from typing import List
names = ['Petr', 'Pavel']
names.append(1)

names: List[str] = []
names.append('Petr')    

<string>:4: error: Argument 1 to "append" of "list" has incompatible type "int"; expected "str"
<string>:6: error: Name 'names' already defined on line 3



## Union

In [26]:
%%typecheck
from typing import List, Union
def norm(vector: List[float], exp: Union[float, str]) -> float:
    if exp == 'inf':
        return max([abs(x) for x in vector])
    else:
        return sum([abs(x)**exp for x in vector])**(1/exp)

print(norm([1,2], 1))
print(norm([1,2], 'inf'))
# print(norm([1,2], 'frobenius'))   # fails

<string>:7: error: Unsupported operand types for ** ("float" and "str")
<string>:7: note: Right operand is of type "Union[float, str]"
<string>:7: error: Unsupported operand types for / ("int" and "str")

3.0
2


## Optional

In [32]:
%%typecheck
from typing import List, Optional
def get_item(vec: List[int], i:int) -> Optional[int]:
    if i < len(vec):
        return vec[i]
    else:
        return None

print(get_item([1,2], 1))
print(get_item([1,2], 3))
get_item([1,2], 1) + 5

<string>:11: error: Unsupported operand types for + ("None" and "int")
<string>:11: note: Left operand is of type "Optional[int]"

2
None


7

## Generic

In [36]:
%%typecheck
from typing import TypeVar

AnyStr = TypeVar('AnyStr', str, bytes)

def concat(a:AnyStr, b:AnyStr) -> AnyStr:
    return a + b

def main()->None:
    concat('x', b'y')
    concat(1, 2)
    reveal_type(concat('x', 'y'))      # processed by 'mypy', undefined in Python
    reveal_type(concat(b'x', b'y'))


<string>:10: error: Value of type variable "AnyStr" of "concat" cannot be "object"
<string>:11: error: Value of type variable "AnyStr" of "concat" cannot be "int"
<string>:12: error: Revealed type is 'builtins.str*'
<string>:13: error: Revealed type is 'builtins.bytes*'



In [52]:
%%typecheck
from typing import TypeVar, List

MatrixItem = TypeVar('MatrixItem')

def mat_col(matrix: List[List[MatrixItem]], i:int) -> List[MatrixItem]:
    return [ row[i] for row in matrix]

def main()->None:
    x_int = mat_col([[1,2], [3,4]], 0)
    reveal_type(sum(x_int))
    x_str = mat_col([['a','b'], ['c','d']], 0)
    reveal_type(' '.join(x_str))


<string>:11: error: Revealed type is 'builtins.int*'
<string>:13: error: Revealed type is 'builtins.str'



## static duck typing

### Nominal subtyping - skrze dědičnost: 

B je potomkem A => B je podtypem A
    
**Structural subtyping** - duck typing

B má všechny atributy, které má A => B je podtypem A

In [42]:
%%typecheck
from typing import Optional, Tuple
from typing_extensions import Protocol

class PersonLike(Protocol):
    name: str
    def have_relation(self, other: 'PersonLike') -> bool: ...

def marriage(a:PersonLike, b:PersonLike) -> Optional[Tuple[str,str]]:
    if a.have_relation(b) and b.have_relation(a):
        return (a.name, b.name)
    else:
        return None

    

## 'Any' - emergency exit

In [45]:
%%typecheck
from typing import Any

class Proxy:
    def __getattr__(self, name:str) -> Any:
        return getattr(self.wrapped, name)

## 'cast' - emergency exit

In [49]:
%%typecheck
from typing import cast

def external_func(val = 0):
    return val

def main() -> None:
    some_value = external_func()
    reveal_type(some_value)
    str_value = cast(str, external_func()) # !! Error
    reveal_type(str_value)



<string>:9: error: Revealed type is 'Any'
<string>:11: error: Revealed type is 'builtins.str'



In [ ]:
## Sequence vs. List, runtime type manipulation

In [61]:
import pytypes

print(pytypes.is_subtype(Tuple[bool], Tuple[int]))
print(pytypes.is_subtype(List[bool], List[int]))
print(pytypes.is_subtype(Sequence[bool], Sequence[int]))


True
False
True


## Další funkce:
-  viz. [Carl Meyer, PyCon 2018](https://www.youtube.com/watch?v=pMgmKJyWKn8)
- @overload - detailnější informace o návratovém typu funkce v závislosti na typu argumentů
- '*.pyi' - něco jako hlavičkový soubor pro externí knihovny
- 'ignore' - vypínání typecheckingu pro části kódu
- gradual typechecking - systém lze pro velké projekty zavádět postupně
- 'monkeytype' - nástroj pro generování 'pyi' pro existující kód
- 'pyre' - rychlejší alternativa k 'mypy'

**realtime kontrola typování v PyCharmu:**

[Daniel Pyrathon (Jet Brains): Putting Type Hints to Work](https://www.youtube.com/watch?v=JqBCFfiE11g)

**podrobný tutoriál:**

[Geir Arne Hjelle: The Ultimate Guide to Python Type Checking](https://realpython.com/python-type-checking/)

# Attrs - knihovna, kterou každý potřebuje

[Attrs documentation](https://www.attrs.org/en/stable/)

Odvozeno od:

[Glyph Lefkowitz: The One Python Library Everyone Needs](https://glyph.twistedmatrix.com/2016/08/attrs.html)





In [74]:
import attr
@attr.s(auto_attribs=True)
class Point:
    x: float
    y: float
        
    def inf_norm(self):
        return abs(self.x) + abs(self.y)

p = Point(1,2)    		# __init__
print(p)          		# __repr__
print(p == Point(1,2))   # __eq__
print(p.inf_norm())
attr.asdict(p)



Point(x=1, y=2)
True
3


{'x': 1, 'y': 2}

In [76]:
q = Point('a', 'b')
print(q)  # __repr__
print(q == Point(1, 2))  # __eq__
print(attr.asdict(q))

Point(x='a', y='b')
False
{'x': 'a', 'y': 'b'}


In [70]:
import dataclasses
# New in Python 3.7

@dataclass
class Point:
    x: float
    y: float
        
    def inf_norm(self):
        return abs(self.x) + abs(self.y)

p = Point(1,2)    		# __init__
print(p)          		# __repr__
print(p == Point(1,2))   # __eq__
print(p.inf_norm())


ModuleNotFoundError: No module named 'dataclasses'

## Srovnání
**namedtuple** - je pouze čitelnější Tuple, nemá nahradit třídy
**dataclasses** - standardizovaná podmnožina 'attrs', chybí:
    	- podpora pro Python 2.7, PyPy
        - validátory, konvertory, __slots__
        